In [2]:
using Distributed
using CPUTime
using DelimitedFiles

In [3]:
addprocs(2)
nprocs()

3

In [4]:
@everywhere using Base
@everywhere using DataStructures
using SharedArrays

In [6]:
numVertex=4
Edges = [(4,1,2), (1,2,1), (2,3,1), (3,4,1), (4,2,3), (1,3,5)]
AdjList = [Vector{}() for _ in 1:numVertex]
for ln in Edges
    push!(AdjList[ln[1]], (ln[1], ln[2],ln[3]))
    push!(AdjList[ln[2]], (ln[2], ln[1],ln[3]))
end


In [8]:

function readDataFromFile(filename="in7.txt")
    open(filename, "r") do f 
        n, m = split(readline(f))
        n = parse(Int, n)
        AdjList = [Vector{}() for _ in 1:n]
#         println(n," ",m)
        for ln in eachline(f)
#             a,b,c = split(ln)
            a,b,c = readdlm(IOBuffer(ln), Int)
            if(a==0)
                a=n
            end
            if(b==0)
                b=n
            end
            push!(AdjList[a], (a,b,c))
            push!(AdjList[b], (b,a,c))
#             println(a," ",b," ",c)
        end
        return n, AdjList
    end
end
numVertex, AdjList = readDataFromFile();

In [18]:
uf = IntDisjointSets(numVertex+5)


IntDisjointSets([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  9996, 9997, 9998, 9999, 10000, 10001, 10002, 10003, 10004, 10005], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 10005)

In [19]:
MAX_INT=2000000000
minCost = SharedArray{Int}(numVertex);
minNodeTgt = SharedArray{Int}(numVertex);
minNodeSrc = SharedArray{Int}(numVertex);

In [20]:
mst= Vector{}()
sizehint!(mst, numVertex)


0-element Array{Any,1}

In [21]:
function initMinCostArray()
    for i = 1:numVertex
        minCost[i] = MAX_INT
        minNodeTgt[i] = i
        minNodeSrc[i] = i
    end
end

initMinCostArray (generic function with 1 method)

In [22]:
function findMinCostVertex(AdjList)
#     for i in 1:length(AdjList)
    @sync @distributed for i in 1:length(AdjList)
        for j in AdjList[i]
#             println(i," -> ", j[1], "=", j[2], " ",  !in_same_set(uf, i, j[1])  )
            set1 = find_root(uf, j[1])
            set2 = find_root(uf, j[2])
            if set1 != set2
                if(minCost[set1] > j[3] )
                    minCost[set1] = j[3]
                    minNodeTgt[set1] = j[2]
                    minNodeSrc[set1] = j[1]
                end
            end
        end
    end
end

findMinCostVertex (generic function with 1 method)

In [23]:
res = 0 
mst = Vector{}()

0-element Array{Any,1}

In [24]:
function contractVertexAndMergeAdjList(AdjList)
    for i in 1:numVertex
        if(minCost[i]!= MAX_INT && !in_same_set(uf, minNodeSrc[i], minNodeTgt[i]))
            # Connect the vertices, add mst to answer
            set1 = find_root(uf, minNodeSrc[i])
            set2 = find_root(uf, minNodeTgt[i])
            union!(uf, minNodeSrc[i], minNodeTgt[i])
            global res+=minCost[i]
            push!(mst, (minNodeSrc[i], minNodeTgt[i], minCost[i]))
            # Merge AdjList 
            len1 = length(AdjList[set1])
            len2 = length(AdjList[set2])
            if (len1> len2)
                for j in AdjList[set2]
                    push!(AdjList[set1],j)
                end
                AdjList[set2] = Vector{}()
            else
                for j in AdjList[set1]
                    push!(AdjList[set2],j)
                    AdjList[set1] = Vector{}()
                end
            end
        end
    end
    return AdjList
end

contractVertexAndMergeAdjList (generic function with 1 method)

In [25]:
function boruvka_MST(AdjList, maxItr = convert(Int64, round(log2(numVertex)+1, digits=0)))
    i=1
    println("Max iteration: ", maxItr)
    while(i<maxItr && length(mst)< numVertex-1)
        i+=1
        println(i, " ", length(mst))
        initMinCostArray()
        findMinCostVertex(AdjList)
        AdjList = contractVertexAndMergeAdjList(AdjList)
        # println(res)
        # println(length(mst))
    end
end

boruvka_MST (generic function with 2 methods)

In [26]:
@time @CPUtime boruvka_MST(AdjList)


Max iteration: 14
2 0
3 7491
4 7491
5 7491
6 7491
7 7491
8 7491
9 7491
10 7491
11 7491
12 7491
13 7491
14 7491
elapsed CPU time: 3.025467 seconds
  5.498324 seconds (18.80 M allocations: 446.772 MiB, 4.54% gc time)


In [ ]:
res

In [ ]:
println(length(mst))